# Recurrent Neural Network Implementation From Scratch

Aim is to train a character level model on The Time Machine...

In [1]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [9]:
class RNNScratch(d2l.Module):
    def __init__(self, num_inputs, num_hiddens, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()

        self.W_xh = nn.Parameter(
            torch.randn(num_inputs, num_hiddens) * sigma
        )
        self.W_hh = nn.Parameter(
            torch.randn(num_hiddens, num_hiddens) * sigma
        )
        self.b_h = nn.Parameter(
            torch.zeros(num_hiddens)
        )

    def forward(self, inputs, state=None):
        if state is None:
            state = torch.zeros((inputs.shape[1], self.num_hiddens), device=inputs.device)

        else:
            state, = state

        outputs = []

        for X in inputs: # Loop over each timestep (character) of the inputs?|??
            state = torch.tanh(torch.matmul(X, self.W_xh) + torch.matmul(state, self.W_hh) + self.b_h)
            outputs.append(state)

        return outputs, state
        

In [10]:
batch_size = 2
num_inputs = 16
num_hiddens = 32
num_steps = 100

In [11]:
rnn = RNNScratch(num_inputs, num_hiddens)
X = torch.ones((num_steps, batch_size, num_inputs))

In [12]:
outputs, state = rnn(X)

## RNN Based Language Model